In [ ]:
from IPython.display import Markdown

import numpy as np
import pandas as pd
import re
from bs4 import BeautifulSoup

from pprint import pprint



**Experimenting automatic issue creation for data cleaning.**

If this doesn't work for you or you are not interested please comment it.

If you want to use the feature,
 - Install Ipython widgets:
         conda install -c conda-forge ipywidgets
         
 - please set your USERNAME and PASSWORD.

In [ ]:
conda install -c conda-forge ipywidgets

In [ ]:
import ipywidgets as widgets
from IPython.display import display
#from ipywidgets import Button, Layout, Textarea, HBox, VBox

import os
import json
import requests

# repository to add issue to)
USERNAME = ''
PASSWORD = ''

REPO_OWNER = 'nathanbollig'
REPO_NAME = 'vet-note-classifier'

#code from https://gist.github.com/JeffPaine/3145490#gistcomment-2226141
def make_github_issue(title, body=None, labels=None):
    '''Create an issue on github.com using the given parameters.'''
    
    if USERNAME=='' or PASSWORD == '':
        print('Error : Please set your USERNAME and PASSWORD...')
        return None
    # Our url to create issues via POST
    url = 'https://api.github.com/repos/%s/%s/issues' % (REPO_OWNER, REPO_NAME)
    # Create an authenticated session to create the issue
    session = requests.Session()
    session.auth = (USERNAME, PASSWORD)
    # Create our issue
    issue = {'title': title,
             'body': body,
             'labels': labels}
    # Add the issue to our repository
    r = session.post(url, json.dumps(issue))
    if r.status_code == 201:
        print ('Successfully created Issue {0:s}'.format(title))
    else:
        print ('Could not create Issue {0:s}'.format(title))
        print ('Response:', r.content)

In [ ]:
data = pd.read_csv('../../data/data_processed/data_processed.csv')
data.index = data.record_number.values

Takes processed dataset as input, it also contains raw HTML.

Prints the first record of sampled patient

In [ ]:
def view_note(record_number = None,print_extracted = False, report = True):
    # sampling unique mr
    sample_mr = np.random.choice(data.mr.values)
    
    #getting the first record for one patient
    i = data.index.values[data.mr == sample_mr][0]
    if record_number:
        i = record_number
    print(f'Record Number # {i}')
    
    if print_extracted:
        print("***"*20)
        print("Hisroty ::")
        print(data.History.loc[i])
        print("\n")
        print("***"*20)
        print("Physical Exam ::")
        print(data.Physical_Exam.loc[i])
        print("\n")
        print("***"*20)
        print("Diagnoses ::")
        print(data.Diagnoses.loc[i])
        print("\n")
        print("***"*20)
        print("Diagnostic_Test ::")
        print(data.Diagnostic_Test.loc[i])
        print("\n")
        print("***"*20)
        print("Cleaned Text ::")
        print(data.Cleaned_Text.loc[i])
        print("\n")
        print(f'Record Number # {i}')
        
        if report:
            button = widgets.Button(description="Report Issue")
            button.style.button_color = 'lightgreen'
            output = widgets.Output()
            l = widgets.Layout(flex='0 1 auto', height='100px', min_height='100px', width='auto')
            
            w = widgets.Textarea(
                value=f'Record Number # {i} ::', #don't remove '::'
                placeholder='',
                description='Issue:',
                disabled=False,
                layout=l
            )
            display(w)
            display(button, output)

            def on_button_clicked(b):
                print(b)
                with output:
                    issue = w.value
                    if issue and len(issue.strip()) > 10:
                        title = issue.split("::")[0]
                        make_github_issue("data cleaning " + title,
                                          issue.split("::")[1],
                                          ['data cleaning'])


            button.on_click(on_button_clicked)

    return Markdown(f'{data.comment_text[i]}')

In [ ]:
view_note(print_extracted=True)